In [3]:
from pyspark.sql import SparkSession

In [9]:
from pyspark.sql.functions import col, to_date, to_timestamp, when
from pyspark.sql.types import StructType, StructField, StringType,  DoubleType, TimestampType,IntegerType

In [4]:
spark = SparkSession.builder.appName("Transforming Food Delivery Data").getOrCreate()

In [10]:
schema = StructType([
    StructField('order_id',IntegerType(),True),
    StructField('customer_id',IntegerType(),True),
    StructField('restaurant_id',IntegerType(),True),
    StructField('order_time',TimestampType(),True),
    StructField('customer_location',StringType(),True),
    StructField('restaurant_location',StringType(),True),
    StructField('order_value',DoubleType(),True),
    StructField('rating',DoubleType(),True),
    StructField('delivery_time',TimestampType(),True)
])

In [14]:
df =spark.read.csv('Orders_data.csv',schema=schema,header=True)

In [15]:
df.show()

+--------+-----------+-------------+-------------------+-----------------+-------------------+-----------+------+-------------------+
|order_id|customer_id|restaurant_id|         order_time|customer_location|restaurant_location|order_value|rating|      delivery_time|
+--------+-----------+-------------+-------------------+-----------------+-------------------+-----------+------+-------------------+
|    9813|       7032|          342|2024-04-06 15:34:13|       15th Floor|           Suite 31|     1810.2|  0.55|2024-04-06 16:31:49|
|    9381|       8269|          541|2024-04-06 12:43:05|         Room 386|           Room 126|     157.01|  4.63|2024-04-06 13:18:59|
|    7804|       3693|          499|2024-04-06 14:54:41|         Suite 67|           Suite 68|     218.23|  0.35|2024-04-06 15:40:03|
|    7726|       2171|          174|2024-04-06 01:22:50|       14th Floor|          Room 1957|    4784.58|  0.35|2024-04-06 01:57:59|
|    6719|       1937|          165|2024-04-06 10:04:25|     P

In [16]:
df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- restaurant_id: integer (nullable = true)
 |-- order_time: timestamp (nullable = true)
 |-- customer_location: string (nullable = true)
 |-- restaurant_location: string (nullable = true)
 |-- order_value: double (nullable = true)
 |-- rating: double (nullable = true)
 |-- delivery_time: timestamp (nullable = true)



In [17]:
df_validated = df.filter(
(df.order_time.isNotNull()) &
(df.delivery_time.isNotNull())&
(df.order_value>0)
)
    

In [18]:
df_transformed = df_validated.withColumn('delivery_duration',
                                         (df_validated.delivery_time - df_validated.order_time).cast('long') / 60)


In [19]:
df_transformed.show(4)

+--------+-----------+-------------+-------------------+-----------------+-------------------+-----------+------+-------------------+-----------------+
|order_id|customer_id|restaurant_id|         order_time|customer_location|restaurant_location|order_value|rating|      delivery_time|delivery_duration|
+--------+-----------+-------------+-------------------+-----------------+-------------------+-----------+------+-------------------+-----------------+
|    9813|       7032|          342|2024-04-06 15:34:13|       15th Floor|           Suite 31|     1810.2|  0.55|2024-04-06 16:31:49|             57.6|
|    9381|       8269|          541|2024-04-06 12:43:05|         Room 386|           Room 126|     157.01|  4.63|2024-04-06 13:18:59|             35.9|
|    7804|       3693|          499|2024-04-06 14:54:41|         Suite 67|           Suite 68|     218.23|  0.35|2024-04-06 15:40:03|45.36666666666667|
|    7726|       2171|          174|2024-04-06 01:22:50|       14th Floor|          Room

In [20]:
low_threshold = 500
high_threshold = 1200

df_transformed = df_transformed.withColumn('order_category',
                                          when(col('order_value')<low_threshold,"Low")
                                          .when((col('order_value')>=low_threshold)& (col('order_value')<=high_threshold),'Medium')
                                          .otherwise("High"))

In [21]:
df_transformed.show(4)

+--------+-----------+-------------+-------------------+-----------------+-------------------+-----------+------+-------------------+-----------------+--------------+
|order_id|customer_id|restaurant_id|         order_time|customer_location|restaurant_location|order_value|rating|      delivery_time|delivery_duration|order_category|
+--------+-----------+-------------+-------------------+-----------------+-------------------+-----------+------+-------------------+-----------------+--------------+
|    9813|       7032|          342|2024-04-06 15:34:13|       15th Floor|           Suite 31|     1810.2|  0.55|2024-04-06 16:31:49|             57.6|          High|
|    9381|       8269|          541|2024-04-06 12:43:05|         Room 386|           Room 126|     157.01|  4.63|2024-04-06 13:18:59|             35.9|           Low|
|    7804|       3693|          499|2024-04-06 14:54:41|         Suite 67|           Suite 68|     218.23|  0.35|2024-04-06 15:40:03|45.36666666666667|           Low